# US Bills Scraping

In [1]:
!pip install striprtf
!pip install openpyxl

In [108]:
import os
import glob
import re
import openpyxl
import requests 
import pandas as pd
import numpy as np 
from functools import reduce
from bs4 import BeautifulSoup
from striprtf.striprtf import rtf_to_text

### Main info

In [88]:
with open("Links.rtf") as infile: 
    content = infile.read()
    links = rtf_to_text(content)

links = links.split('\n')

In [89]:
links_all = []
for link in links:
    links_all += re.findall('\(("[^)]*")\)', link)

links_all = [s.replace('"', "") for s in links_all]

In [90]:
links_all

['https://www.govtrack.us/congress/bills/117/s4201',
 'https://www.govtrack.us/congress/bills/117/s1896',
 'https://www.govtrack.us/congress/bills/117/hr3611',
 'https://www.govtrack.us/congress/bills/117/hr83',
 'https://www.govtrack.us/congress/bills/117/s3663',
 'https://www.govtrack.us/congress/bills/117/s5259',
 'https://www.govtrack.us/congress/bills/117/s673',
 'https://www.govtrack.us/congress/bills/117/s4200',
 'https://www.govtrack.us/congress/bills/117/s3898',
 'https://www.govtrack.us/congress/bills/117/s2572',
 'https://www.govtrack.us/congress/bills/117/s1204',
 'https://www.govtrack.us/congress/bills/117/hr4634',
 'https://www.govtrack.us/congress/bills/117/hr1735',
 'https://www.govtrack.us/congress/bills/117/s1384',
 'https://www.govtrack.us/congress/bills/117/s2427',
 'https://www.govtrack.us/congress/bills/117/hr7703',
 'https://www.govtrack.us/congress/bills/117/s1127',
 'https://www.govtrack.us/congress/bills/117/hr7819',
 'https://www.govtrack.us/congress/bills/11

In [91]:
def parse(links):
    data = []
    counter = 0

    for link in links: 
        r = requests.get(link)
        link_soup = BeautifulSoup(r.content)
        
        try: 
            data.append({"code": link_soup.select("li.active")[0].text,
                "title": link_soup.select("div.h1-multiline")[0].text,
                "date": link_soup.select("div.col-sm-9.col-md-10")[0].text,
                "status": link_soup.select("div.col-sm-9.col-md-10")[1].find('strong').text,
                "sponsor": link_soup.select("a.name")[0].text,
                "affiliation": link_soup.select("div.col-xs-7.col-sm-4")[0].find('p').text,
                "# co-sponsor/s": link_soup.select("div.col-sm-9.col-md-10")[2].find('a').text, 
                "summary": link_soup.select("div.col-sm-8.col-sm-pull-4")[0].find('p', style="margin-bottom: 0").text,
                "link": str(link)})
        except:
            pass

        counter += 1
        if (counter) % 20 == 0:
            print("No. links scraped: " + str(counter))

    raw = pd.DataFrame.from_dict(data)
    return raw
    #raw.to_csv('raw.csv')

In [112]:
df = parse(links_all)

No. links scraped: 20
No. links scraped: 40
No. links scraped: 60
No. links scraped: 80
No. links scraped: 100


In [113]:
len(df)

106

In [114]:
df_main = (df
    #.drop(df.columns[0], axis =1)
    .applymap(lambda x: x.strip() if type(x)==str else x))

In [115]:
df_main

,code,title,date,status,sponsor,affiliation,# co-sponsor/s,summary,link
0,S. 4201 (117th),S. 4201 (117th): Digital Platform Commission A...,"May 12, 2022\n\t\t\t\t\n\t\t\t\t117th Congress...",Died in a previous Congress,Michael Bennet,Sponsor. Senator for Colorado. Democrat.,Congress.gov,A bill to establish a new Federal body to prov...,https://www.govtrack.us/congress/bills/117/s4201
1,S. 1896 (117th),S. 1896 (117th): Algorithmic Justice and Onlin...,"May 27, 2021\n\t\t\t\t\n\t\t\t\t117th Congress...",Died in a previous Congress,Edward “Ed” Markey,Sponsor. Junior Senator for Massachusetts. Dem...,3 Cosponsors,A bill to prohibit the discriminatory use of p...,https://www.govtrack.us/congress/bills/117/s1896
2,H.R. 3611 (117th),H.R. 3611 (117th): Algorithmic Justice and Onl...,"May 28, 2021\n\t\t\t\t\n\t\t\t\t117th Congress...",Died in a previous Congress,Doris Matsui,Sponsor. Representative for California's 6th c...,11 Cosponsors,To prohibit the discriminatory use of personal...,https://www.govtrack.us/congress/bills/117/hr3611
3,H.R. 83 (117th),H.R. 83 (117th): Protecting Constitutional Rig...,"Jan 4, 2021\n\t\t\t\t\n\t\t\t\t117th Congress ...",Died in a previous Congress,Scott DesJarlais,Sponsor. Representative for Tennessee's 4th co...,Congress.gov,To amend the Communications Act of 1934 to pro...,https://www.govtrack.us/congress/bills/117/hr83
4,S. 3663 (117th),S. 3663 (117th): Kids Online Safety Act,"Feb 16, 2022\n\t\t\t\t\n\t\t\t\t117th Congress...",Died in a previous Congress,Richard Blumenthal,Sponsor. Senator for Connecticut. Democrat.,13 Cosponsors,A bill to protect the safety of children on th...,https://www.govtrack.us/congress/bills/117/s3663
...,...,...,...,...,...,...,...,...,...
101,H.R. 6531 (114th),H.R. 6531 (114th): National Aeronautics and Sp...,"Dec 20, 2016\n\t\t\t\t\n\t\t\t\t114th Congress...",Died in a previous Congress,Lamar Smith,Sponsor. Representative for Texas's 21st congr...,Congress.gov,To authorize the programs of the National Aero...,https://www.govtrack.us/congress/bills/114/hr6531
102,H.R. 1806 (114th),H.R. 1806 (114th): America COMPETES Reauthoriz...,"Apr 15, 2015\n\t\t\t\t\n\t\t\t\t114th Congress...",Died in a previous Congress,Lamar Smith,Sponsor. Representative for Texas's 21st congr...,10 Cosponsors,To provide for technological innovation throug...,https://www.govtrack.us/congress/bills/114/hr1806
103,H.R. 2039 (114th),H.R. 2039 (114th): National Aeronautics and Sp...,"Apr 28, 2015\n\t\t\t\t\n\t\t\t\t114th Congress...",Died in a previous Congress,Steven Palazzo,Sponsor. Representative for Mississippi's 4th ...,16 Cosponsors,To authorize the programs of the National Aero...,https://www.govtrack.us/congress/bills/114/hr2039
104,H.R. 1561 (114th),H.R. 1561 (114th): Weather Research and Foreca...,"Mar 24, 2015\n\t\t\t\t\n\t\t\t\t114th Congress...",Died in a previous Congress,Frank Lucas,Sponsor. Representative for Oklahoma's 3rd con...,6 Cosponsors,To improve the National Oceanic and Atmospheri...,https://www.govtrack.us/congress/bills/114/hr1561


In [117]:
df_main.date = df_main.date.str.extract(r'([a-zA-Z]{3,}\s*[0-9]{2,}\s*[,\s]\s*[0-9]{2,})') #extract date based on MMM-DD-YYYY format
df_main.title = df_main.title.str.extract(r'([^:]*$)') #remove code from title 

In [118]:
df_main['# co-sponsor/s'] = ['None' if x == 'Congress.gov' else x for x in df_main['# co-sponsor/s']] 
df_main['affiliation'] = [x.replace("Sponsor. ", "") for x in df_main['affiliation']]

In [120]:
df_main

,code,title,date,status,sponsor,affiliation,# co-sponsor/s,summary,link
0,S. 4201 (117th),Digital Platform Commission Act of 2022,"May 12, 2022",Died in a previous Congress,Michael Bennet,Senator for Colorado. Democrat.,None,A bill to establish a new Federal body to prov...,https://www.govtrack.us/congress/bills/117/s4201
1,S. 1896 (117th),Algorithmic Justice and Online Platform Trans...,"May 27, 2021",Died in a previous Congress,Edward “Ed” Markey,Junior Senator for Massachusetts. Democrat.,3 Cosponsors,A bill to prohibit the discriminatory use of p...,https://www.govtrack.us/congress/bills/117/s1896
2,H.R. 3611 (117th),Algorithmic Justice and Online Platform Trans...,"May 28, 2021",Died in a previous Congress,Doris Matsui,Representative for California's 6th congressio...,11 Cosponsors,To prohibit the discriminatory use of personal...,https://www.govtrack.us/congress/bills/117/hr3611
3,H.R. 83 (117th),Protecting Constitutional Rights from Online ...,NaN,Died in a previous Congress,Scott DesJarlais,Representative for Tennessee's 4th congression...,None,To amend the Communications Act of 1934 to pro...,https://www.govtrack.us/congress/bills/117/hr83
4,S. 3663 (117th),Kids Online Safety Act,"Feb 16, 2022",Died in a previous Congress,Richard Blumenthal,Senator for Connecticut. Democrat.,13 Cosponsors,A bill to protect the safety of children on th...,https://www.govtrack.us/congress/bills/117/s3663
...,...,...,...,...,...,...,...,...,...
101,H.R. 6531 (114th),National Aeronautics and Space Administration...,"Dec 20, 2016",Died in a previous Congress,Lamar Smith,Representative for Texas's 21st congressional ...,None,To authorize the programs of the National Aero...,https://www.govtrack.us/congress/bills/114/hr6531
102,H.R. 1806 (114th),America COMPETES Reauthorization Act of 2015,"Apr 15, 2015",Died in a previous Congress,Lamar Smith,Representative for Texas's 21st congressional ...,10 Cosponsors,To provide for technological innovation throug...,https://www.govtrack.us/congress/bills/114/hr1806
103,H.R. 2039 (114th),National Aeronautics and Space Administration...,"Apr 28, 2015",Died in a previous Congress,Steven Palazzo,Representative for Mississippi's 4th congressi...,16 Cosponsors,To authorize the programs of the National Aero...,https://www.govtrack.us/congress/bills/114/hr2039
104,H.R. 1561 (114th),Weather Research and Forecasting Innovation A...,"Mar 24, 2015",Died in a previous Congress,Frank Lucas,Representative for Oklahoma's 3rd congressiona...,6 Cosponsors,To improve the National Oceanic and Atmospheri...,https://www.govtrack.us/congress/bills/114/hr1561


In [121]:
df_main.to_excel('US Bills-01.xlsx')

### Co-sponsors

In [100]:
sponsors_links = [x + "/cosponsors" for x in links_all]

In [103]:
def parse_sponsors(links):
    sponsor_data = []
    counter = 0

    for link in links: 
        r = requests.get(link)
        link_soup = BeautifulSoup(r.content)

        for table in link_soup.find_all('table'):
            table = link_soup.find('table', id='cosponsors')

            temp = []
            for row in table.tbody.find_all('tr'):    
                columns = row.find_all('td')
                if(columns != []):
                    name = columns[0].text.strip()
                    sponsorship = columns[1].text.strip()
                    #committee = columns[2].text.strip()

                    temp.append({"name":name, "sponsorship":sponsorship})

            sponsor_df = pd.DataFrame.from_dict(temp)

            sponsor_df['affiliation'] = [re.findall('(?<=\[).+?(?=\])', x)[0] for x in sponsor_df['name']]
            sponsor_df['name'] = [re.findall('[A-Za-z]+,\s[A-Za-z\s]+', x)[0].strip() for x in sponsor_df['name']]

            co_sponsor_name = sponsor_df[(sponsor_df['sponsorship'] != "Primary Sponsor")]['name']
            co_sponsor_aff = sponsor_df[(sponsor_df['sponsorship'] != "Primary Sponsor")]['affiliation']

        sponsor_data.append({"cosponsors": '; '.join(co_sponsor_name),
                            "cosponsors_affiliation": '; '.join(co_sponsor_aff),
                            "link": str(link).replace("/cosponsors", "")})

        counter += 1
        if (counter) % 20 == 0:
            print("No. links scraped: " + str(counter))

    sponsors = pd.DataFrame.from_dict(sponsor_data)
    return sponsors

In [104]:
sponsors = parse_sponsors(sponsors_links)

In [105]:
sponsors

,cosponsors,cosponsors_affiliation,link
0,,,https://www.govtrack.us/congress/bills/117/s4201
1,"Whitehouse, Sheldon; Warren, Elizabeth; Booker...",D-RI; D-MA; D-NJ,https://www.govtrack.us/congress/bills/117/s1896
2,"Eshoo, Anna; Clarke, Yvette; Deutch, Theodore;...",D-CA18; D-NY9; D-FL22; D-MI13; D-MD7; D-CA11; ...,https://www.govtrack.us/congress/bills/117/hr3611
3,,,https://www.govtrack.us/congress/bills/117/hr83
4,"Blackburn, Marsha; Capito, Shelley; Markey, Ed...",R-TN; R-WV; D-MA; D-NM; D-WI; D-MN; D-MI; D-CO...,https://www.govtrack.us/congress/bills/117/s3663
...,...,...,...
104,,,https://www.govtrack.us/congress/bills/114/hr6531
105,"Lucas, Frank; Babin, Brian; Weber, Randy; Coms...",R-OK3; R-TX36; R-TX14; R-VA10; R-IL14; R-CA25;...,https://www.govtrack.us/congress/bills/114/hr1806
106,"Lucas, Frank; Babin, Brian; Posey, Bill; Weber...",R-OK3; R-TX36; R-FL8; R-TX14; R-OK1; R-AL5; R-...,https://www.govtrack.us/congress/bills/114/hr2039
107,"Bonamici, Suzanne; Bridenstine, Jim; Johnson, ...",D-OR1; R-OK1; D-TX30; R-CA48; R-TX21; R-UT2,https://www.govtrack.us/congress/bills/114/hr1561


Sample sponsors dataframe

In [39]:
sample = "https://www.govtrack.us/congress/bills/117/s673/cosponsors"

In [40]:
r = requests.get(sample)
s = BeautifulSoup(r.content)

In [6]:
for table in s.find_all('table'):
    print(table.get('id'))

cosponsors
None


In [7]:
table = s.find('table', id='cosponsors')

In [8]:
temp = []

for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')

    if(columns != []):
        name = columns[0].text.strip()
        sponsorship = columns[1].text.strip()
        #committee = columns[2].text.strip()

        temp.append({"name":name, "sponsorship":sponsorship})

sponsor_df = pd.DataFrame.from_dict(temp)

In [ ]:
temp = []

for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')

    if(columns != []):
        name = columns[0].text.strip()
        sponsorship = columns[1].text.strip()
        #committee = columns[2].text.strip()

        temp.append({"name":name, "sponsorship":sponsorship})

sponsor_df = pd.DataFrame.from_dict(temp)

sponsor_df['affiliation'] = [re.findall('(?<=\[).+?(?=\])', x)[0] for x in sponsor_df['name']]
sponsor_df['name'] = [re.findall('[A-Za-z]+,\s[A-Za-z\s]+', x)[0].strip() for x in sponsor_df['name']]

co_sponsor_name = sponsor_df[(sponsor_df['sponsorship'] != "Primary Sponsor")]['name']
co_sponsor_aff = sponsor_df[(sponsor_df['sponsorship'] != "Primary Sponsor")]['affiliation']

cosponsors = '; '.join(co_sponsor_name)
cosponsors_aff = '; '.join(co_sponsor_aff)


In [10]:
sponsor_df

,name,sponsorship,affiliation
0,"Klobuchar, Amy",Primary Sponsor,D-MN
1,"Durbin, Richard","Aug 6, 2022",D-IL
2,"Graham, Lindsey","Mar 10, 2022",R-SC
3,"Booker, Cory",Original Cosponsor,D-NJ
4,"Kennedy, John Neely",Original Cosponsor,R-LA
5,"Whitehouse, Sheldon",Original Cosponsor,D-RI
6,"Feinstein, Dianne","Jun 24, 2021",D-CA
7,"Hirono, Mazie","Jul 13, 2022",D-HI
8,"Blumenthal, Richard","Aug 6, 2022",D-CT
9,"Paul, Rand",Original Cosponsor,R-KY


In [38]:
'; '.join(co_sponsor_name)

'Durbin, Richard; Graham, Lindsey; Booker, Cory; Kennedy, John Neely; Whitehouse, Sheldon; Feinstein, Dianne; Hirono, Mazie; Blumenthal, Richard; Paul, Rand; Collins, Susan; Lummis, Cynthia; Cassidy, Bill; Thune, John; Manchin, Joe; Wicker, Roger'

In [35]:
'; '.join(co_sponsor_aff)

'D-IL; R-SC; D-NJ; R-LA; D-RI; D-CA; D-HI; D-CT; R-KY; R-ME; R-WY; R-LA; R-SD; D-WV; R-MS'

### Committee

In [67]:
comm_links = [x + "/details" for x in links_all]

In [85]:
def parse_comms(links):
    comm_data = []
    counter = 0

    for link in links: 
        r = requests.get(link)
        link_soup = BeautifulSoup(r.content)
        
        try: 
            comm_data.append({"committee": link_soup.find_all('div', style = 'line-height: 125%;')[0].text,
                "link": str(link).replace("/details", "")})
        except:
            pass

        counter += 1
        if (counter) % 20 == 0:
            print("No. links scraped: " + str(counter))

    comms = pd.DataFrame.from_dict(comm_data)
    return comms

In [86]:
comms = parse_comms(comm_links)

No. links scraped: 20
No. links scraped: 40
No. links scraped: 60
No. links scraped: 80
No. links scraped: 100


In [106]:
comms

,committee,link
0,"Senate Commerce, Science, and Transportation",https://www.govtrack.us/congress/bills/117/s4201
1,"Senate Commerce, Science, and Transportation",https://www.govtrack.us/congress/bills/117/s1896
2,House Energy and Commerce,https://www.govtrack.us/congress/bills/117/hr3611
3,House Energy and Commerce,https://www.govtrack.us/congress/bills/117/hr83
4,"Senate Commerce, Science, and Transportation",https://www.govtrack.us/congress/bills/117/s3663
...,...,...
104,"House Science, Space, and Technology",https://www.govtrack.us/congress/bills/114/hr6531
105,House Education and the Workforce,https://www.govtrack.us/congress/bills/114/hr1806
106,"House Science, Space, and Technology",https://www.govtrack.us/congress/bills/114/hr2039
107,"House Science, Space, and Technology",https://www.govtrack.us/congress/bills/114/hr1561


### Merge

In [122]:
dfs = [df_main, sponsors, comms]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['link'], how='outer'), dfs).fillna(' ')

In [123]:
df_merged

,code,title,date,status,sponsor,affiliation,# co-sponsor/s,summary,link,cosponsors,cosponsors_affiliation,committee
0,S. 4201 (117th),Digital Platform Commission Act of 2022,"May 12, 2022",Died in a previous Congress,Michael Bennet,Senator for Colorado. Democrat.,None,A bill to establish a new Federal body to prov...,https://www.govtrack.us/congress/bills/117/s4201,,,"Senate Commerce, Science, and Transportation"
1,S. 1896 (117th),Algorithmic Justice and Online Platform Trans...,"May 27, 2021",Died in a previous Congress,Edward “Ed” Markey,Junior Senator for Massachusetts. Democrat.,3 Cosponsors,A bill to prohibit the discriminatory use of p...,https://www.govtrack.us/congress/bills/117/s1896,"Whitehouse, Sheldon; Warren, Elizabeth; Booker...",D-RI; D-MA; D-NJ,"Senate Commerce, Science, and Transportation"
2,H.R. 3611 (117th),Algorithmic Justice and Online Platform Trans...,"May 28, 2021",Died in a previous Congress,Doris Matsui,Representative for California's 6th congressio...,11 Cosponsors,To prohibit the discriminatory use of personal...,https://www.govtrack.us/congress/bills/117/hr3611,"Eshoo, Anna; Clarke, Yvette; Deutch, Theodore;...",D-CA18; D-NY9; D-FL22; D-MI13; D-MD7; D-CA11; ...,House Energy and Commerce
3,H.R. 83 (117th),Protecting Constitutional Rights from Online ...,,Died in a previous Congress,Scott DesJarlais,Representative for Tennessee's 4th congression...,None,To amend the Communications Act of 1934 to pro...,https://www.govtrack.us/congress/bills/117/hr83,,,House Energy and Commerce
4,S. 3663 (117th),Kids Online Safety Act,"Feb 16, 2022",Died in a previous Congress,Richard Blumenthal,Senator for Connecticut. Democrat.,13 Cosponsors,A bill to protect the safety of children on th...,https://www.govtrack.us/congress/bills/117/s3663,"Blackburn, Marsha; Capito, Shelley; Markey, Ed...",R-TN; R-WV; D-MA; D-NM; D-WI; D-MN; D-MI; D-CO...,"Senate Commerce, Science, and Transportation"
...,...,...,...,...,...,...,...,...,...,...,...,...
104,H.R. 1561 (114th),Weather Research and Forecasting Innovation A...,"Mar 24, 2015",Died in a previous Congress,Frank Lucas,Representative for Oklahoma's 3rd congressiona...,6 Cosponsors,To improve the National Oceanic and Atmospheri...,https://www.govtrack.us/congress/bills/114/hr1561,"Bonamici, Suzanne; Bridenstine, Jim; Johnson, ...",D-OR1; R-OK1; D-TX30; R-CA48; R-TX21; R-UT2,"House Science, Space, and Technology"
105,H.R. 6490 (114th),American Innovation and Competitiveness Act,,Enacted Via Other Measures,Lamar Smith,Representative for Texas's 21st congressional ...,None,To invest in innovation through research and d...,https://www.govtrack.us/congress/bills/114/hr6490,,,House Education and the Workforce
106,,,,,,,,,https://www.govtrack.us/congress/bills/116/s893,"Blackburn, Marsha; Sullivan, Dan; Bennet, Mich...",R-TN; R-AK; D-CO; R-NC; R-ME; R-AR; D-CA; R-FL...,"Senate Commerce, Science, and Transportation"
107,,,,,,,,,https://www.govtrack.us/congress/bills/116/s2661,"Baldwin, Tammy; Moran, Jerry; Tester, Jon; Fis...",D-WI; R-KS; D-MT; R-NE; I-AZ; R-IN; R-TN; D-MA...,"Senate Commerce, Science, and Transportation"


In [124]:
df_merged.to_excel('US Bills-01-complete.xlsx')